In [39]:
from argparse import Namespace 
import torch
import numpy as np
from torch.utils.data import DataLoader
from src.detector.data_obj_detect import MOT16ObjDetect
from src.detector.utils import obj_detect_transforms

args = Namespace()
args.data_root_dir = "data/MOT16/train"
args.vis_threshold = 0.25
args.num_workers = 0
args.batch_size = 2

dataset_test = MOT16ObjDetect(
        root=args.data_root_dir,
        transforms=obj_detect_transforms(train=False),
        vis_threshold=args.vis_threshold,
        segmentation=True
    )

data_loader_test = DataLoader(
    dataset_test,
    batch_size=args.batch_size,
    shuffle=False,
    num_workers=args.num_workers,
    collate_fn=lambda batch: tuple(zip(*batch)),
)

In [40]:
images, targets = next(iter(data_loader_test))

In [34]:
from PIL import Image
import torchvision.transforms.functional as TF
import torch
import torchvision
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from src.tracker.data_track_precomputed import MOT16SequencesPrecomputed
from src.utils.torch_utils import dict2keys_and_items

sequences = MOT16SequencesPrecomputed(
        precomputed_data_root_dir="data/precomputed_detection/fasterrcnn",
        original_data_root_dir="data/MOT16",
        split="train",
        vis_threshold=0.0,
)


sequence = sequences[0]
frame = sequence[0]


fig, ax = plt.subplots(figsize=(15, 15))
ax.imshow(frame["img"].permute(1, 2, 0))
for box in frame["boxes"]:
    x, y, w, h = torchvision.ops.box_convert(box, "xyxy", "xywh")
    ax.add_patch(Rectangle((x, y), w, h, fill=False))

# for id, box in frame["gt"].items():
#     x, y, w, h = torchvision.ops.box_convert(torch.from_numpy(box), "xyxy", "xywh")
#     ax.add_patch(Rectangle((x, y), w, h, fill=False))



In [25]:

# write existing database to my folder structure
import os
from src.utils.file_utils import ensure_dir
from src.tracker.data_track import MOT16Sequence
from tqdm import tqdm
db = torch.load("data/precomputed_detection/chair/preprocessed_data_train_2.pth")
dir = "data/precomputed_detection/default"

for seq_name in tqdm(db.keys(), desc="seqs"):
    dets = []
    for frame_id, frame in enumerate(db[seq_name], start=1):
        det = {"boxes": frame["det"]["boxes"], "scores": frame["det"]["scores"]}
        dets.append(det)
        reid_on_gt = frame["gt"]["reid"]
        reid_on_det = frame["det"]["reid"]
        reid_on_gt_path = os.path.join(dir, seq_name, "reid", "reid_on_gt", f"{frame_id:06d}.pth")
        reid_on_det_path = os.path.join(dir, seq_name, "reid", "reid_on_det", f"{frame_id:06d}.pth")
        ensure_dir(reid_on_gt_path)
        ensure_dir(reid_on_det_path)
        torch.save(reid_on_gt, reid_on_gt_path)
        torch.save(reid_on_det, reid_on_det_path)

    det_path = os.path.join(dir, seq_name, "detection.pth")
    ensure_dir(det_path)
    torch.save(dets, det_path)

        
    

seqs: 100%|██████████| 7/7 [00:07<00:00,  1.10s/it]


In [14]:
# create new detector
import json
import torch
from src.detector.object_detector import init_detector_from_config

obj_detect_path = "config/obj_detect/maskrcnn.json"
save_path = "models/obj_detect/default_maskrcnn.pth"
with open(obj_detect_path, "r") as f:
    config = json.load(f)

obj_detect = init_detector_from_config(config)
torch.save(obj_detect, save_path)

In [24]:
# create new assign model
import json
import torch
from src.gnn.assignment_model import AssignmentWeightedAverage

config_path = "config/assign_model/mask_and_box_iou.json"
save_path = "models/assign_model/mask_and_box_iou.pth"
with open(config_path, "r") as f:
    config = json.load(f)

assign_model = AssignmentWeightedAverage.from_config(config)
torch.save(assign_model, save_path)

In [76]:
not not {}

False

In [50]:
import pandas as pd
import json
import numpy as np
import torch
import argparse
import matplotlib.pyplot as plt
import torchvision
from matplotlib.patches import Rectangle
from src.tracker.tracker import MyTracker
from src.tracker.data_track_precomputed import MOT16SequencesPrecomputed
from src.tracker.data_track import load_detection_from_txt

txt_path = "results\\tracker\\06-04-2022_23-53\\tracker_predictions_MOT16-mini\\MOT16-02-mini.txt"
det = load_detection_from_txt(txt_path, vis_threshold=0.0, mode="track")


df = pd.read_csv("events.csv")
with open("results\\tracker\\06-04-2022_23-08\\eval_config.json", "r") as f:
    eval_config = json.load(f)
args = argparse.Namespace(**eval_config)
dataset = f"MOT16-{args.split}"
with open(args.tracker_config_path, "r") as f:
    tracker_hyperparams = json.load(f)

# execute
tracker = MyTracker.from_config(tracker_hyperparams)
sequences = MOT16SequencesPrecomputed(
        precomputed_data_root_dir=args.precomputed_data_root_dir,
        original_data_root_dir=args.original_data_root_dir,
        split=args.split,
        vis_threshold=args.vis_threshold,
        return_det_segmentation=tracker.assign_model.use_segmentation,
        return_gt_segmentation=tracker.assign_model.use_segmentation,
)
sequence = sequences[0]

In [31]:
frame_id = 1
prev_frame_df = df.groupby("FrameId").get_group(frame_id - 1)
frame_df = df.groupby("FrameId").get_group(frame_id)

match_df = frame_df.groupby("Type").get_group("MATCH")
oids = match_df["OId"].astype(int).tolist()
hids = match_df["HId"].astype(int).tolist()

In [36]:
len(det_boxes) == len(gt_boxes)

True

In [60]:
img = sequence[frame_id]["img"]
matched_det_boxes = [det["boxes"][frame_id][hid + 1] for hid in hids]
matched_gt_boxes = [sequence[frame_id]["gt"][oid] for oid in oids]

fig, ax = plt.subplots(figsize=(10, 10))
plt.axis('off')

ax.imshow(img.permute(1, 2, 0))
for det_box, gt_box in zip(matched_det_boxes, matched_gt_boxes):
    gt_box = torchvision.ops.box_convert(torch.from_numpy(gt_box), "xyxy", "xywh")
    det_box = torchvision.ops.box_convert(torch.from_numpy(det_box), "xyxy", "xywh")
    gt_rect = Rectangle(gt_box[:2], gt_box[2], gt_box[3], fill=False, color="r")
    det_rect = Rectangle(det_box[:2], det_box[2], det_box[3], fill=False, color="g")
    ax.add_patch(gt_rect)
    ax.add_patch(det_rect)
